# Imports

In [1]:
import os
import re
import csv
import sys
# import unicodedata
# from unidecode import unidecode

# Pre Processing
How you want the text to be read from the file. Add additional changes you want to make to the file before trying the NLP. 
       The first one joins all groups of consecutive lines together, so

        Doe, John. Columbia University
        12345 N. 1st St.
        BA, 1921

        Miller, Sue. Columbia University
        12345 N. 2nd St.
        BS, 1922

        ----->

        Doe, John. Columbia University 12345 N. 1st St. BA, 1921
        Miller, Sue. Columbia University 12345 N. 2nd St. BS, 1922

That makes writing a regex easier if your .txt files look like the above. If they already look like the output of this function, then you can run the other pre-process cell instead.
   

In [2]:
def pre_process(fname):
    with open(fname, encoding='utf-8') as fin:
        lines = re.sub(r'(.)\-\n(.)', lambda x : x.group(1) + x.group(2), fin.read()).split('\n')
    return re.sub(r'([^\n])\n', lambda x : x.group(1) + ' ', '\n'.join(lines))

### An alternate pre-processing function that just reads lines. Only run one of these!

In [5]:
def pre_process(fname):
    with open(fname, encoding='utf-8') as fin:
        lines = fin.read()
    return lines

# Function that makes lists of people

In [3]:
def collect(text):
    """Collects all names and returns a formatted list of matches from text<str>."""  
    if type(text) is not str:
        raise NotImplementedError

    # This loop looks through lines (from above) and appends each match to a list, formatted the way we want
    matches = []
    non_matches = []
    
    for line in text.split('\n'):
        i = people_re.search(line)
        if i is not None:
            matches.append((i.groupdict()['first'].strip() + ' ' + i.groupdict()['last'].strip(), i.groupdict()['info']))
#             matches.append((i.groupdict()['first'], i.groupdict()['last'], i.groupdict()['class'], i.groupdict()['info']))
#             matches.append(tuple(i.groupdict().values()))
        else:
            non_matches.append(line)
    
    return matches, non_matches


# Define the regex and which school to search
Go to https://pythex.org/ to test your regex.

In [6]:
people_re = re.compile(r'(?P<last>([a-z]+)), (?P<first>[a-z ]+) (?P<info>.*)', flags=re.IGNORECASE)

# Name of the school
source = "Columbia"

# Name of the csv file to write to
target = f"NLP_Output_{source}.csv"

# This is where the non-matched lines will be written
chk_file = 'check.csv'

os.chdir(r'..\output\University CSVs\{}'.format(source))

# Run this just to make sure that if you rerun the code,
# it makes a new file instead of appending

In [4]:
# Update target file name so that we aren't appending to an existing file
    
if os.path.exists(target):
    i = 1
    name, ext = target.split('.')
    while os.path.exists(f'{name}_{i}.{ext}'):
        i += 1
    target = f'{name}_{i}.{ext}'

if os.path.exists(chk_file):
    i = 1
    name, ext = chk_file.split('.')
    while os.path.exists(f'{name}_{i}.{ext}'):
        i += 1
    chk_file = f'{name}_{i}.{ext}'

NameError: name 'target' is not defined

# Main function
If you interrupt this cell or the previous one and the next run gives an OS error, restart the kernel and then try again.

In [5]:
os.chdir(r'..\..\University Text Files\{}'.format(source))
for txt in [i for i in os.listdir() if i[-4:] == '.txt']:
    # This is where the work happens. Uses the collect() function.
    print(f'finding names in {txt}...')
    result, check = collect(pre_process(txt))
    num = len(result)
    print(f'found {num} names. Writing to file {target}...')
    os.chdir(r'..\..\University CSVs\{}'.format(source))

    # Write matches to the target file (.csv)
    with open(target, 'a') as fout:
        writer = csv.writer(fout)
        writer.writerow(['Name', 'Address(es)'])#, 'State', 'School', 'Additional Degrees', 'Year found'])

        # Write the names, addresses, and year found
        for match in result:
            writer.writerow([i for i in match] + [txt[-8:-4]])

        # Total names found that year
        writer.writerow(['Names found:', num])

    # Write non-matches to the check file
    with open(chk_file, 'a') as fout:
        writer = csv.writer(fout)
        for i in check:
            try:
                writer.writerow([i])
            except UnicodeEncodeError:
                writer.writerow(["Error on line"])

    os.chdir(r'..\..\University Text Files\{}'.format(source))

NameError: name 'source' is not defined